# Longitudinal Data Analysis

# Extensions

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Nonlinear-outcomes" data-toc-modified-id="Nonlinear-outcomes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Nonlinear outcomes</a></span></li><li><span><a href="#Hybrid-panel-data-models" data-toc-modified-id="Hybrid-panel-data-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Hybrid panel data models</a></span></li><li><span><a href="#Mundlak-approach" data-toc-modified-id="Mundlak-approach-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Mundlak approach</a></span></li><li><span><a href="#Dynamic-panel-models" data-toc-modified-id="Dynamic-panel-models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dynamic panel models</a></span></li><li><span><a href="#Latent-growth-curve-models" data-toc-modified-id="Latent-growth-curve-models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Latent growth curve models</a></span></li></ul></div>

In this section we provide a whistle-stop tour of some additional techniques and approaches for panel data and longitudinal data more broadly.

## Nonlinear outcomes

Fixed Effects and Random Effects models can be applied to nonlinear outcomes (e.g., binary and count dependent variables) also.

Here is a published example from McDonnell (2017): [https://doi.org/10.1177/0899764017692039](https://doi.org/10.1177/0899764017692039)

In [1]:
use "../data/improvingcharityaccountability_20170411.dta", clear

gen localc = (geographicalspread==2)
gen linc = ln(totalfunds) if totalfunds > 0 & totalfunds!=.


(Scottish Charity Financial Exceptions Data: 2007-2013)


(1,323 missing values generated)


In [2]:
tab yearsubmitted excgroup_3 


      Year |  Possible failure to
    annual |    apply funds for
    return |  charitable purposes
 submitted |         0          1 |     Total
-----------+----------------------+----------
      2007 |       754        196 |       950 
      2008 |     2,752        881 |     3,633 
      2009 |     2,964        818 |     3,782 
      2010 |     2,946        736 |     3,682 
      2011 |     2,659        702 |     3,361 
      2012 |     2,450        645 |     3,095 
      2013 |     1,555        457 |     2,012 
      2014 |       585        222 |       807 
-----------+----------------------+----------
     Total |    16,665      4,657 |    21,322 



In [3]:
xtlogit excgroup_3 concentration charityage localc linc, or re


Fitting comparison model:

Iteration 0:   log likelihood = -10687.029  
Iteration 1:   log likelihood = -10488.084  
Iteration 2:   log likelihood = -10486.442  
Iteration 3:   log likelihood = -10486.442  

Fitting full model:

tau =  0.0     log likelihood = -10486.442
tau =  0.1     log likelihood = -10257.949
tau =  0.2     log likelihood =  -10048.82
tau =  0.3     log likelihood = -9859.3094
tau =  0.4     log likelihood =  -9689.005
tau =  0.5     log likelihood = -9538.3489
tau =  0.6     log likelihood = -9410.3167
tau =  0.7     log likelihood = -9314.0716
tau =  0.8     log likelihood =  -9277.005

Iteration 0:   log likelihood = -9313.6924  
Iteration 1:   log likelihood = -9178.5858  
Iteration 2:   log likelihood = -9173.5625  
Iteration 3:   log likelihood = -9173.5365  
Iteration 4:   log likelihood = -9173.5365  (backed up)
Iteration 5:   log likelihood = -9173.5362  

Random-effects logistic regression              Number of obs     =     19,982
Group variable: org_i

In [4]:
use "../data/charity-panel-analysis-2020-09-10.dta", clear

xtpoisson nsources linc orgage localc west genchar govern_share, re


(Contains annual accounts of charities in E&W for financial years 2006-2017)


Fitting Poisson model:

Iteration 0:   log likelihood =  -42473.77  
Iteration 1:   log likelihood =  -42473.77  

Fitting full model:

Iteration 0:   log likelihood = -43378.386  
Iteration 1:   log likelihood = -41912.848  (not concave)
Iteration 2:   log likelihood = -41494.954  
Iteration 3:   log likelihood = -41471.918  
Iteration 4:   log likelihood = -41471.687  
Iteration 5:   log likelihood = -41471.687  

Random-effects Poisson regression               Number of obs     =     23,826
Group variable: regno                           Number of groups  =      2,166

Random effects u_i ~ Gamma                      Obs per group:
                                                              min =         11
                                                              avg =       11.0
                                                              max =         11

                                        

## Hybrid panel data models

A hybrid panel model allows you to decompose the observed explanatory variables into their within and between effects using the Random Effects estimator.

Let's return to our charity data example and see if we can decompose the effect of `nsources` into its within and between effects.

In [5]:
use "../data/charity-panel-analysis-2020-09-10.dta", clear

(Contains annual accounts of charities in E&W for financial years 2006-2017)


In [6]:
bys regno: egen nsources_mn = mean(nsources)
gen nsources_delta = nsources - nsources_mn

In [7]:
xtreg linc orgage localc west genchar nsources_mn nsources_delta govern_share, re


Random-effects GLS regression                   Number of obs     =     23,826
Group variable: regno                           Number of groups  =      2,166

R-sq:                                           Obs per group:
     within  = 0.0136                                         min =         11
     between = 0.1017                                         avg =       11.0
     overall = 0.0952                                         max =         11

                                                Wald chi2(7)      =     536.49
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

-------------------------------------------------------------------------------
         linc |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
       orgage |   .0048981   .0003692    13.27   0.000     .0041745    .0056216
       localc |  -.3320839   .0412748    -8.05   0.000     -

The coefficients for `nsources_mn` and `nsources_delta` are equal to those estimated in the Between Effects and Fixed Effects models respectively.

Furthermore we can test whether the between and within effects are equal:

In [8]:
test nsources_mn = nsources_delta


 ( 1)  nsources_mn - nsources_delta = 0

           chi2(  1) =   28.78
         Prob > chi2 =    0.0000


An equivalent approach is to use the `mundlak` command:

In [9]:
mundlak linc orgage localc west genchar nsources govern_share, hybrid


The variable orgage does not vary sufficiently within groups and will not be use
> d to create additional regressors.
0% of the total variance in orgage is within groups.

The variable localc does not vary sufficiently within groups and will not be use
> d to create additional regressors.
0% of the total variance in localc is within groups.

The variable west does not vary sufficiently within groups and will not be used 
> to create additional regressors.
0% of the total variance in west is within groups.

The variable genchar does not vary sufficiently within groups and will not be us
> ed to create additional regressors.
0% of the total variance in genchar is within groups.

+------------------------------------------------+
|             Variable |     RE     |   Hybrid   |
|----------------------+------------+------------|
|               orgage |      0.005 |      0.005 |
|               localc |     -0.332 |     -0.329 |
|                 west |      0.080 |      0.097 |
|      

## Mundlak approach

Random Effects model assumes that observed and unobserved effects are uncorrelated - an often unrealistic assumption (Gayle and Lambert, 2018).

We can relax this assumption using the *Mundlak approach*, which works by including unit-level means for the time-varying explanatory variables in the Random Effects model.

In [10]:
bys regno: egen orgage_mn = mean(orgage)
bys regno: egen govern_share_mn = mean(govern_share)

In [11]:
xtreg linc orgage localc west genchar nsources govern_share ///
    govern_share_mn nsources_mn orgage_mn, re
est store mund



Random-effects GLS regression                   Number of obs     =     23,826
Group variable: regno                           Number of groups  =      2,166

R-sq:                                           Obs per group:
     within  = 0.0140                                         min =         11
     between = 0.1042                                         avg =       11.0
     overall = 0.0976                                         max =         11

                                                Wald chi2(9)      =     557.99
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

-------------------------------------------------------------------------------
         linc |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
       orgage |   .0069072   .0005802    11.90   0.000       .00577    .0080444
       localc |  -.3282906   .0414364    -7.92   0.000    -

In [12]:
quietly xtreg linc orgage localc west genchar nsources govern_share, fe
est store fixed

In [13]:
est table fixed mund


----------------------------------------
    Variable |   fixed         mund     
-------------+--------------------------
      orgage |   .0069072     .0069072  
      localc |  (omitted)    -.3282906  
        west |  (omitted)    .11673923  
     genchar |  (omitted)   -.32109178  
    nsources |  .02898861    .02898861  
govern_share |  .00103247    .00103247  
govern_sha~n |              -.00075753  
 nsources_mn |               .10947099  
   orgage_mn |              -.00340245  
       _cons |  14.715042    14.850581  
----------------------------------------


In [14]:
quietly xtreg linc orgage localc west genchar nsources govern_share ///
    govern_share_mn nsources_mn orgage_mn, re

test govern_share_mn = nsources_mn = orgage_mn




 ( 1)  govern_share_mn - nsources_mn = 0
 ( 2)  govern_share_mn - orgage_mn = 0

           chi2(  2) =   43.79
         Prob > chi2 =    0.0000


The *Mundlak approach* is an alternative to the *Hausman test*.

## Dynamic panel models

The models are suitable for when you have repeated contacts data and your (lagged) outcome variable serves also serves as one of your explanatory variables.

The inclusion of lagged outcome variables poses as an issue as the lagged variables are possibly correlated with the unobserved effects (Gayle and Lambert, 2018).

In [15]:
use "../data/charity-panel-analysis-2020-09-10.dta", clear
xtset regno fin_year


(Contains annual accounts of charities in E&W for financial years 2006-2017)

       panel variable:  regno (strongly balanced)
        time variable:  fin_year, 1 to 11
                delta:  1 unit


In [16]:
capture gen linc_lag = L.linc
l regno fin_year linc linc_lag in 1/22, clean




        regno   fin_year       linc   linc_lag  
  1.   200048    2006-07   14.00189          .  
  2.   200048    2007-08   14.17788   14.00189  
  3.   200048    2008-09    14.1851   14.17788  
  4.   200048    2009-10    14.2326    14.1851  
  5.   200048    2010-11    14.1709    14.2326  
  6.   200048    2011-12   14.14801    14.1709  
  7.   200048    2012-13     14.376   14.14801  
  8.   200048    2013-14   14.29996     14.376  
  9.   200048    2014-15   14.26031   14.29996  
 10.   200048    2015-16   14.30113   14.26031  
 11.   200048    2016-17   14.37021   14.30113  
 12.   200051    2006-07     17.664          .  
 13.   200051    2007-08   17.60568     17.664  
 14.   200051    2008-09   17.44065   17.60568  
 15.   200051    2009-10   16.46766   17.44065  
 16.   200051    2010-11   16.32526   16.46766  
 17.   200051    2011-12    16.4079   16.32526  
 18.   200051    2012-13   16.35779    16.4079  
 19.   200051    2013-14   16.04346   16.35779  
 20.   200051    

In [17]:
xtreg linc orgage localc west genchar nsources govern_share linc_lag, re


Random-effects GLS regression                   Number of obs     =     21,660
Group variable: regno                           Number of groups  =      2,166

R-sq:                                           Obs per group:
     within  = 0.2673                                         min =         10
     between = 0.9963                                         avg =       10.0
     overall = 0.9320                                         max =         10

                                                Wald chi2(7)      =  296585.19
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

------------------------------------------------------------------------------
        linc |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      orgage |  -.0000262   .0000438    -0.60   0.549     -.000112    .0000596
      localc |  -.0065562   .0038027    -1.72   0.085    -.01400

Note how large the coefficient is for the lagged variable (and how much smaller the others have become). This is a common issue when including lagged outcome variables as one of the explanatory variables i.e., the lagged variable soaks up all of the variation accounted for by the unobserved unit-specific effects. 

In [18]:
xtreg linc orgage localc west genchar nsources govern_share linc_lag, fe

note: localc omitted because of collinearity
note: west omitted because of collinearity
note: genchar omitted because of collinearity

Fixed-effects (within) regression               Number of obs     =     21,660
Group variable: regno                           Number of groups  =      2,166

R-sq:                                           Obs per group:
     within  = 0.2705                                         min =         10
     between = 0.9695                                         avg =       10.0
     overall = 0.9098                                         max =         10

                                                F(4,19490)        =    1807.17
corr(u_i, Xb)  = 0.9009                         Prob > F          =     0.0000

------------------------------------------------------------------------------
        linc |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      org

A set of dynamic panel models &mdash; commonly known as *Arrelano-Bond* models &mdash; have been developed to address the inclusion of a lagged outcome as an explanatory variable.

They also have the advantage of controlling for "initial conditions".

That is, data collection sometimes interrupts an ongoing social process, and thus the outcome observed at the first time point is partially accounted for factors not measured at first time point (Gayle and Lambert, 2018).

## Latent growth curve models

Statistical modelling of repeated contacts data.

Focuses on trajectory, trend or growth in an outcome over time **within** units.

And how these trajectories are linked to observed and unobserved differences **between** units.

Latent growth curve models can be estimated using a *Multilevel modelling* framework &mdash; random intercepts, random slopes.

They can also be estimated using a *Structural Equation Modelling (SEM)* framework &mdash;  there exists underlying continuous trajectory of change that is not directly observed.

**Honesty time**

[Not an area I know a great deal about - see the reading list for suggested resources]